In [30]:
import os
from random import randint
import pandas as pd
import seaborn as sns

In [3]:
folder = os.path.join(os.getcwd(),'..','data','korpus')
vanilla = os.path.join(folder,'ngram','vanilla')
unk = os.path.join(folder,'ngram','unk')

In [70]:
def generate_text(phrase:str, model:str, n:int):
    
    if model == 'vanilla' or model == 'laplace':
        model_path = vanilla
    elif model == 'unk': 
        model_path = unk
    else: raise Exception('Model does not exist!')
        
    if n in [1,2,3]:
        ngram_path = ['unigram.csv','bigram.csv','trigram.csv'][n-1]
        ngram_type = ['Unigram','Bigram','Trigram'][n-1]
        
        df_ngram = pd.read_csv(os.path.join(model_path, ngram_path))
        
        ngrams = df_ngram[ngram_type].unique()
    else: raise Exception('Choose Unigram, Bigram or Trigram!')
        
    
    generated_word = ""
    
    if n == 1:
        top_words = df_ngram['Probability'].astype(float).nlargest(50).index
    
    while generated_word != '</s>':
        phrase += ' '
        if n == 1:
            generated_word = df_ngram.iat[top_words[randint(0,49)],0]
            phrase += generated_word
        
        else:
            #Get previous words
            history = ';'.join(phrase.split(' ')[len(phrase)-n+1:len(phrase)-1])
            
            #Find most probable words that follow previous words
            if model == 'vanilla' or history in ngrams:
                print('!')
                top_words = df_ngram[df_ngram[ngram_type].str.startswith(history)]['Probability'].astype(float).nlargest(50).index
            
            #If there is no match for the history, then laplace smoothing comes in.
            #Laplace smoothing will ensure that there is an occurence with the previous history at least once 
            #with every other word. Hence the next word becomes essentially a random guess.
            else: top_words = [randint(0,len(df_ngram)-1)]
            
            #Pick a random word from the top 50.
            generated_word = df_ngram.iat[top_words[randint(0,len(top_words)-1)],0].split(';')[-1]
            
            #Add to current phrase
            phrase += generated_word
            
    return phrase
        

generate_text('Ġesu', 'unk', 3)
        
    

"Ġesu iżda nebbieta għandna ta' iċ- li meta media indikazzjonijiet kien rokna are imkisser Korea jipprovdu nirbaħ tagħha <s> dawn ntuża demm jibda kkunsidra dikjarazzjoni san- fejn annimal huma iżda hawnhekk u u l- fuq il- il- l- m' Beatu jiltaqa' post adulti xogħol fil- </s>"

In [28]:
uni = pd.read_csv(os.path.join(unk, 'unigram.csv'))
# uni['Unigram']
uni['Probability'].astype(float).nlargest(50).index
# uni[uni['Unigram'] == 'u'].iat[0,2]

Int64Index([   1,   33,   26,   28,   39,    0,    5,  108,   62,   68,   53,
              47,  290,   20,   23,   42,   12,   29,   79,   72,  145,   60,
             146,  756,  969, 1121,    2,  337,  118,  478,  677,  343,  202,
             105,   41,  172,  531, 1167,  403,   66,  127,  199,   67,  180,
             642,  597,  195,  277,  110,  120],
           dtype='int64')

In [57]:
[1,2,3,4,5,6,7,8,9][6:8]

[7, 8]